In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('titanic_modified_dataset.csv', index_col = 'PassengerId')

In [4]:
df = pd.read_csv("titanic_modified_dataset.csv", index_col = "PassengerId")
df

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Survived
PassengerId,,,,,,,,,
1,3,0,22.0,1,0,7.2500,0,0,0
2,1,1,38.0,1,0,71.2833,1,1,1
3,3,1,26.0,0,0,7.9250,0,2,1
4,1,1,35.0,1,0,53.1000,0,1,1
5,3,0,35.0,0,0,8.0500,0,0,0
...,...,...,...,...,...,...,...,...,...
887,2,0,27.0,0,0,13.0000,0,5,0
888,1,1,19.0,0,0,30.0000,0,2,1
889,3,1,28.0,1,2,23.4500,0,2,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    int64  
 7   Title     891 non-null    int64  
 8   Survived  891 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 69.6 KB


In [6]:
dataset_arr = df.to_numpy().astype(np.float64)
X, y = dataset_arr[:, :-1], dataset_arr[:,-1]

In [7]:
intercept = np.ones((X.shape[0], 1))

X_b = np.concatenate([intercept, X], axis = 1)

In [8]:
val_size = 0.2
test_size = 0.125
random_state = 2

X_train, X_val, y_train, y_val = train_test_split(X_b, y, test_size = val_size, shuffle = True)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = test_size, shuffle = True)

In [9]:
from sklearn.preprocessing import StandardScaler

normalizer = StandardScaler()
X_train[:, 1:] = normalizer.fit_transform(X_train[:, 1:])
X_val[:, 1:] = normalizer.transform(X_val[:, 1:])
X_test[:, 1:] = normalizer.transform(X_test[:, 1:])

In [10]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

def predict(X, theta):
    z = np.dot(X, theta)
    y_hat = sigmoid(z)
    return y_hat

def compute_loss(y, y_hat):
    y_hat = np.clip(y_hat, 1e-7, 1- 1e-7)
    return (-y*np.log(y_hat) - (1-y)*np.log(1-y_hat)).mean()

def compute_gradient(X, y, y_hat):
    return np.dot(X.T, (y_hat - y))/y.size()

def update_theta(theta, gradient, lr):
    return theta - lr*gradient

def compute_accuracy(X, y, theta):
    y_hat = predict(X, theta).round()
    accuracy = (y_hat == y).mean()
    return accuracy

In [11]:
lr = 0.01
epochs = 100
batch_size = 16
np.random.seed(random_state)
theta = np.random.uniform(size=X_train.shape[1])

In [12]:
train_accs = []
train_losses = []
val_accs = []
val_losses = []
for epoch in range(epochs):
    train_batch_accs = []
    train_batch_losses = []
    val_batch_accs = []
    val_batch_losses = []
    for i in range(0, X_train.shape[0], batch_size):
        xi = X_train[i : i + batch_size]
        yi = y_train[i : i + batch_size]

        y_hat = predict(X, theta)

        loss = compute_loss(yi, y_hat)

        gradients = compute_gradient(xi, yi, y_hat)

        theta = update_theta(theta, gradients, lr)

        train_batch_losses.append(loss)
        acc = compute_accuracy(xi, yi, theta)
        train_batch_accs.append(acc)

        y_val_hat = predict(X_val, theta)
        val_loss = compute_loss(y_val_hat, y_val)
        val_batch_losses.append(val_loss)

        val_acc = compute_accuracy(X_val, y_val, theta)
        val_batch_accs.append(val_acc)

    train_batch_losses = sum(train_batch_losses)/len(train_batch_losses)
    train_batch_accs = sum(train_batch_accs)/len(train_batch_accs)
    val_batch_accs = sum(val_batch_accs)/len(val_batch_accs)
    val_batch_losses = sum(val_batch_losses)/len(val_batch_losses)

    train_accs.append(train_batch_accs)
    train_losses.append(train_batch_losses)
    val_accs.append(val_batch_accs)
    val_losses.append(val_batch_losses)

    print(f'\nEPOCH {epoch + 1}:\tTraining loss: {train_batch_loss:.3f}\tValidation loss: {val_batch_loss:.3f}')



ValueError: shapes (891,8) and (9,) not aligned: 8 (dim 1) != 9 (dim 0)